In [1]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [3]:
ticker_list = ["aapl", "amzn", "msft"]

ticker = ticker_list[0]

ticker_upper = ticker.upper()

print(ticker_upper)

AAPL


In [4]:
alphavantage_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}.csv"

In [5]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date_time"] = pd.to_datetime(df['date'])

df["day_of_week"] = df["date_time"].dt.dayofweek

df["adjusted_close_shift"] = df["5. adjusted close"].shift(-1)

df["percentage_change"] = (df["adjusted_close_shift"] - df["5. adjusted close"]) / df["5. adjusted close"] * 100.0

df["increase"] = df["percentage_change"].apply(lambda x: 1 if x > 0 else 0)

df.set_index("date", inplace=True)

In [6]:
df.shape[0]

3064

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               3064 non-null   float64       
 1   2. high               3064 non-null   float64       
 2   3. low                3064 non-null   float64       
 3   4. close              3064 non-null   float64       
 4   5. adjusted close     3064 non-null   float64       
 5   6. volume             3064 non-null   float64       
 6   7. dividend amount    3064 non-null   float64       
 7   8. split coefficient  3064 non-null   float64       
 8   date_time             3064 non-null   datetime64[ns]
 9   day_of_week           3064 non-null   int64         
 10  adjusted_close_shift  3063 non-null   float64       
 11  percentage_change     3063 non-null   float64       
 12  increase              3064 non-null   int64         
dtypes: datet

In [8]:
df.tail()
# df[df.percentage_change == 0]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase
date,,,,,,,,,,,,,
2022-02-28,163.060,165.42,162.43,165.12,165.12,95056629.0,0.0,1.0,2022-02-28,0,163.20,-1.162791,0
2022-03-01,164.695,166.60,161.97,163.20,163.20,83474425.0,0.0,1.0,2022-03-01,1,166.56,2.058824,1
2022-03-02,164.390,167.36,162.95,166.56,166.56,79724750.0,0.0,1.0,2022-03-02,2,166.23,-0.198127,0
2022-03-03,168.470,168.91,165.55,166.23,166.23,76678441.0,0.0,1.0,2022-03-03,3,163.17,-1.840823,0
2022-03-04,164.490,165.55,162.10,163.17,163.17,83819592.0,0.0,1.0,2022-03-04,4,NaN,NaN,0


In [9]:
df.increase.value_counts()

1    1621
0    1443
Name: increase, dtype: int64

In [10]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday
df.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [11]:
loc = df.index.get_loc('2010-01-05')
loc

print(loc)

df.iloc[loc]["date_time"].strftime("%Y-%m-%d")

1


'2010-01-05'

In [12]:
twint_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker}_finbert_twint_20100101_20220304.csv"

In [13]:
twint_csv_path

'/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/AAPL/aapl_finbert_twint_20100101_20220304.csv'

In [14]:
df_twint = pd.read_csv(twint_csv_path, low_memory=False, lineterminator='\n')

In [15]:
df_twint.head()

,Headline,Stock,Positive,Negative,Neutral,Retweets
0,Apple 2.0: Tablet: Big iPhone or thin MacBook?,2010-01-01,0.999959,3.629326e-05,0.000005,7
1,"Apple 2.0: Apple tablet to ship in March, sources say",2010-01-04,0.999997,1.389081e-07,0.000003,6
2,"Apple Tablet Will Be 10-11 Inches, Ships In March, Says WSJ by @jwyarow",2010-01-04,0.999986,8.988271e-06,0.000005,9
3,CHART OF THE DAY: Android Taking Wind Out Of iPhone's Sails $GOOG $RIMM,2010-01-04,0.999992,3.523099e-06,0.000004,7
4,Apple 2.0: How many iPhones did Apple sell?,2010-01-04,0.999959,1.397303e-05,0.000027,10


In [16]:
df_twint['date'] = pd.to_datetime(df_twint['Stock'])

In [17]:
df_twint["day_of_week"] = df_twint["date"].dt.dayofweek

In [18]:
df_twint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41056 entries, 0 to 41055
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     41032 non-null  object        
 1   Stock        41056 non-null  object        
 2   Positive     41056 non-null  float64       
 3   Negative     41056 non-null  float64       
 4   Neutral      41056 non-null  float64       
 5   Retweets     41056 non-null  int64         
 6   date         41056 non-null  datetime64[ns]
 7   day_of_week  41056 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 2.5+ MB


In [19]:
df_twint.head(1)

,Headline,Stock,Positive,Negative,Neutral,Retweets,date,day_of_week
0,Apple 2.0: Tablet: Big iPhone or thin MacBook?,2010-01-01,0.999959,0.000036,0.000005,7,2010-01-01,4


In [20]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

df_twint.day_of_week.value_counts()

1    7676
0    7512
2    7467
3    6485
4    6359
6    2817
5    2740
Name: day_of_week, dtype: int64

In [21]:
df_twint.head(1)

,Headline,Stock,Positive,Negative,Neutral,Retweets,date,day_of_week
0,Apple 2.0: Tablet: Big iPhone or thin MacBook?,2010-01-01,0.999959,0.000036,0.000005,7,2010-01-01,4


In [22]:
df_twint["in_index"] = df_twint["Stock"].apply(lambda x: True if x in df.index else False)

In [23]:
df_twint[df_twint.in_index==False]["day_of_week"].value_counts()

6    2817
5    2740
0    346 
4    109 
1    54  
3    46  
2    24  
Name: day_of_week, dtype: int64

In [24]:
df_twint[df_twint.in_index==True]["day_of_week"].value_counts()

1    7622
2    7443
0    7166
3    6439
4    6250
Name: day_of_week, dtype: int64

In [25]:
df.index.get_loc('2010-01-04')


0

In [26]:
from datetime import date, datetime, timedelta

In [27]:
test2 =  '2010-01-09'

dtobj1 = datetime.strptime(test2, "%Y-%m-%d")

print(dtobj1)

days = timedelta(2)

print(days)



dtobj2 = (dtobj1 - days).strftime("%Y-%m-%d")
dtobj2

2010-01-09 00:00:00
2 days, 0:00:00


'2010-01-07'

In [28]:
def update_date(x):

    global ERROR_COUNTER

    if x.in_index == True:
        return x.Stock
    else:
        # try:

            dt_time = datetime.strptime(x.Stock, "%Y-%m-%d")

            # loc = df.index.get_loc(date_part)
            if x.day_of_week == 6: # Sunday to Thursday
                days = timedelta(3)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e1:
                    ERROR_COUNTER += 1
                    print(str(e1))
                    return x.Stock
            if x.day_of_week == 5: # Saturday to Thursday
                days = timedelta(2)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e2:
                    ERROR_COUNTER += 1
                    print(str(e2))
                    return x.Stock
            if x.day_of_week == 4: # Friday to Thursday
                days = timedelta(1)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock
            if x.day_of_week == 0: # Monday to Thursday
                days = timedelta(4)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(e3))
                    return x.Stock

            else:
                return x.Stock

        # except Exception as e:
        #     ERROR_COUNTER += 1
        #     print(str(e))
        #     print(x.day_of_week, x.Stock, x.in_index)
        #     return x.Stock
    # loc = df.index.get_loc('2010-01-01')
    # loc



In [29]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

ERROR_COUNTER = 0 

df_twint["backfill_date"] = df_twint["Stock"] ## df_twint.apply(update_date, axis=1)


In [30]:
df_twint.head()

,Headline,Stock,Positive,Negative,Neutral,Retweets,date,day_of_week,in_index,backfill_date
0,Apple 2.0: Tablet: Big iPhone or thin MacBook?,2010-01-01,0.999959,3.629326e-05,0.000005,7,2010-01-01,4,False,2010-01-01
1,"Apple 2.0: Apple tablet to ship in March, sources say",2010-01-04,0.999997,1.389081e-07,0.000003,6,2010-01-04,0,True,2010-01-04
2,"Apple Tablet Will Be 10-11 Inches, Ships In March, Says WSJ by @jwyarow",2010-01-04,0.999986,8.988271e-06,0.000005,9,2010-01-04,0,True,2010-01-04
3,CHART OF THE DAY: Android Taking Wind Out Of iPhone's Sails $GOOG $RIMM,2010-01-04,0.999992,3.523099e-06,0.000004,7,2010-01-04,0,True,2010-01-04
4,Apple 2.0: How many iPhones did Apple sell?,2010-01-04,0.999959,1.397303e-05,0.000027,10,2010-01-04,0,True,2010-01-04


In [31]:
df_twint["in_index"] = df_twint["backfill_date"].apply(lambda x: True if x in df.index else False)

In [32]:
df_twint[df_twint.in_index==False]["day_of_week"].value_counts()

6    2817
5    2740
0    346 
4    109 
1    54  
3    46  
2    24  
Name: day_of_week, dtype: int64

In [33]:
df_twint[df_twint.in_index==True].shape

(34920, 10)

In [34]:
# df_benzinga["text"] = df_benzinga["title"] + " " + df_benzinga["body"]

In [35]:
# df_benzinga['text'] = np.where(df_benzinga["text"], df_benzinga["title"], df_benzinga["text"])

In [36]:
# df_twint = df_twint[["backfill_date", "Positive", "Negative", "Neutral", "Retweets"]]
df_twint = df_twint[["backfill_date", "Positive", "Negative", "Neutral"]]

In [37]:
df_twint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41056 entries, 0 to 41055
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   backfill_date  41056 non-null  object 
 1   Positive       41056 non-null  float64
 2   Negative       41056 non-null  float64
 3   Neutral        41056 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1.3+ MB


In [38]:
df_twint.head()

,backfill_date,Positive,Negative,Neutral
0,2010-01-01,0.999959,3.629326e-05,0.000005
1,2010-01-04,0.999997,1.389081e-07,0.000003
2,2010-01-04,0.999986,8.988271e-06,0.000005
3,2010-01-04,0.999992,3.523099e-06,0.000004
4,2010-01-04,0.999959,1.397303e-05,0.000027


In [39]:
df_twint.rename(columns={"backfill_date": "date"}, inplace=True)

In [40]:
def clean_tweet(df):
    # replace URLs with a whitespace
    df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    
    return df

In [41]:
# df_twint = clean_tweet(df_twint)

In [42]:
df_twint.set_index("date", inplace=True)

In [43]:
df_twint.head()

,Positive,Negative,Neutral
date,,,
2010-01-01,0.999959,3.629326e-05,0.000005
2010-01-04,0.999997,1.389081e-07,0.000003
2010-01-04,0.999986,8.988271e-06,0.000005
2010-01-04,0.999992,3.523099e-06,0.000004
2010-01-04,0.999959,1.397303e-05,0.000027


In [44]:
df_twint_duplicated_index = df_twint[df_twint.index.duplicated(keep=False)]

In [45]:
df_twint_duplicated_index.shape[0]

40779

In [46]:
df_twint_nonduplicated_index = df_twint[~df_twint.index.duplicated(keep=False)]

In [47]:
df_twint_nonduplicated_index.shape[0]

277

In [48]:
# df_twint.groupby('date')['negative'].mean()

df_twint_groupby = df_twint.groupby("date").agg(
     negative = ("Negative", "mean"),
     nuetral = ("Neutral", "mean"),
     positive = ("Positive", "mean"),
     # retweets = ("Retweets", "sum"),
          # compound = ("compound", "mean"),
          # nlikes = ("nlikes", "sum"),
          # nretweets = ("nretweets", "sum"),
     )


In [49]:
df_twint_groupby

,negative,nuetral,positive
date,,,
2010-01-01,0.000036,0.000005,0.999959
2010-01-04,0.000007,0.000010,0.999984
2010-01-05,0.498715,0.001116,0.500169
2010-01-06,0.000052,0.085701,0.914247
2010-01-07,0.457010,0.000224,0.542766
...,...,...,...
2022-02-28,0.102984,0.000162,0.896855
2022-03-01,0.083351,0.250679,0.665971
2022-03-02,0.150388,0.000115,0.849497


In [50]:
df_merge = pd.merge(df, df_twint_groupby, how="inner", left_index=True, right_index=True)

In [51]:
df_merge.shape

(2942, 16)

In [52]:
df_merge.day_of_week.value_counts()

2    607
1    605
4    588
3    587
0    555
Name: day_of_week, dtype: int64

In [53]:
# df.index
# df_text.index
df_merge[df_merge["5. adjusted close"].isnull()]
# df_merge.shape

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,


In [54]:
df.shape

(3064, 13)

In [55]:
df_merge.shape

(2942, 16)

In [56]:
df.shape

(3064, 13)

In [57]:
df_merge[df_merge["negative"].isnull()]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,


In [58]:
# df_merge.fillna(method="ffill", inplace=True)

In [59]:
df_merge[df_merge.negative.isnull()]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,


In [60]:
 #df_merge.fillna(0.0, inplace=True)

In [61]:
# df_merge.to_csv(f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}_WITH_TWINT_FINBERT_SA.csv")
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,6.543876,17633200.0,0.0,1.0,2010-01-04,0,6.555190,0.172889,1,0.000007,0.000010,0.999984
2010-01-05,214.60,215.59,213.25,214.38,6.555190,21496600.0,0.0,1.0,2010-01-05,1,6.450921,-1.590633,0,0.498715,0.001116,0.500169
2010-01-06,214.38,215.23,210.75,210.97,6.450921,19720000.0,0.0,1.0,2010-01-06,2,6.438996,-0.184860,0,0.000052,0.085701,0.914247
2010-01-07,211.75,212.00,209.05,210.58,6.438996,17040400.0,0.0,1.0,2010-01-07,3,6.481804,0.664830,1,0.457010,0.000224,0.542766
2010-01-08,210.30,212.00,209.06,211.98,6.481804,15986100.0,0.0,1.0,2010-01-08,4,6.424624,-0.882159,0,0.010157,0.000059,0.989783


In [62]:
# df_merge = df_merge.sample(frac=1).reset_index(drop=True)

In [63]:
# X = df_merge[["negative", "nuetral", "positive", "retweets"]]
X = df_merge[["negative", "nuetral", "positive"]]
y = df_merge["increase"]

In [64]:
y.head()

date
2010-01-04    1
2010-01-05    0
2010-01-06    0
2010-01-07    1
2010-01-08    0
Name: increase, dtype: int64

In [65]:
SPLIT = int(0.9 * len(df_merge))

X_train = X[:SPLIT]
X_test = X[SPLIT:]

y_train = y[:SPLIT]
y_test = y[SPLIT:]



In [66]:
X_train

,negative,nuetral,positive
date,,,
2010-01-04,0.000007,0.000010,0.999984
2010-01-05,0.498715,0.001116,0.500169
2010-01-06,0.000052,0.085701,0.914247
2010-01-07,0.457010,0.000224,0.542766
2010-01-08,0.010157,0.000059,0.989783
...,...,...,...
2020-12-14,0.012838,0.000224,0.986938
2020-12-15,0.215641,0.085961,0.698398
2020-12-16,0.214189,0.127090,0.658722


In [67]:
from sklearn.ensemble import RandomForestClassifier

In [68]:
rfc = RandomForestClassifier(n_estimators=200, criterion="entropy")
rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [69]:
preds = rfc.predict(X_test)

In [70]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [71]:
matrix = confusion_matrix(y_test, preds)

In [72]:
matrix

array([[67, 75],
       [76, 77]])

In [73]:
score = accuracy_score(y_test, preds)

In [74]:
score

0.488135593220339

In [75]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.47      0.47      0.47       142
           1       0.51      0.50      0.50       153

    accuracy                           0.49       295
   macro avg       0.49      0.49      0.49       295
weighted avg       0.49      0.49      0.49       295



In [76]:
# corr_matrix = df_merge[["negative", "nuetral", "positive", "retweets", "increase"]].corr()
corr_matrix = df_merge[["negative", "nuetral", "positive", "increase"]].corr()
print (corr_matrix)

          negative   nuetral  positive  increase
negative  1.000000 -0.174434 -0.713740  0.014146
nuetral  -0.174434  1.000000 -0.565172 -0.047997
positive -0.713740 -0.565172  1.000000  0.022290
increase  0.014146 -0.047997  0.022290  1.000000


In [77]:
from statsmodels.tsa.stattools import grangercausalitytests

In [78]:
#perform Granger-Causality test
grangercausalitytests(df_merge[["positive", "increase"]], maxlag=[63])


Granger Causality
number of lags (no zero) 63
ssr based F test:         F=0.8645  , p=0.7678  , df_denom=2752, df_num=63
ssr based chi2 test:   chi2=56.9740 , p=0.6897  , df=63
likelihood ratio test: chi2=56.4175 , p=0.7082  , df=63
parameter F test:         F=0.8645  , p=0.7678  , df_denom=2752, df_num=63


{63: ({'ssr_ftest': (0.8644554245732667, 0.767834987764773, 2752.0, 63),
   'ssr_chi2test': (56.97395768271272, 0.6897212083919158, 63),
   'lrtest': (56.41754353230499, 0.7082259543673997, 63),
   'params_ftest': (0.8644554245732632, 0.7678349877647833, 2752.0, 63.0)},
   array([[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 1., 0., 0.],
          [0., 0., 0., ..., 0., 1., 0.]])])}